**This notebook will push some inputs into the model created, and use the inputs and the derived prices to plot the summaries on a Folium map**

In [1]:
import pickle
import json
import branca
import folium
import pandas as pd
from module2_scripts.pricer import Pricer

In [2]:
#import model, input code mapping, and binning configurations 
model = pickle.load(open("pkl/model.p", "rb"))
mapping = json.load(open('module2_configs/column_code_config.jsonc', 'r'))
bin_config = json.load(open('module2_configs/manual_binning_config.jsonc', 'r'))

In [3]:
def df_to_html(row):
    
    #function to convert pandas dataframe rows into html
    #purpose is to be able to display inputs in folium marker popups
    i = row    
    zipcode = df['Zipcode'].iloc[i]                             
    bedrooms = df['Bedrooms'].iloc[i]                           
    bathrooms = df['Bathrooms'].iloc[i]
    sqft_living = df['Home Square Footage'].iloc[i]                                           
    waterfront = 'Yes' if df['Waterfront'].iloc[i] == 1 else 'No'
    floors = df['Floors'].iloc[i]  
    latitude = df['Latitude'].iloc[i]   
    longitude = df['Longitude'].iloc[i]   
    price = df['Derived Price'].iloc[i]   
    
    left_col_colour = "#2A799C"
    right_col_colour = "#C5DCE7"
    
    html = """
        <!DOCTYPE html>
        <html>

        <head>
        <h4 style="margin-bottom:0"; width="300px">{}</h4>""".format('Summary') + """

        </head>
            <table style="height: 126px; width: 300px;">
        <tbody>
        <tr>
        <td style="background-color: """+ left_col_colour +""";"><span style="color: #ffffff;">Zipcode</span></td>
        <td style="width: 200px;background-color: """+ right_col_colour +""";">{}</td>""".format(zipcode) + """
        </tr>
        <tr>
        <td style="background-color: """+ left_col_colour +""";"><span style="color: #ffffff;">Bedrooms</span></td>
        <td style="width: 200px;background-color: """+ right_col_colour +""";">{}</td>""".format(bedrooms) + """
        </tr>
        <tr>
        <td style="background-color: """+ left_col_colour +""";"><span style="color: #ffffff;">Bathrooms</span></td>
        <td style="width: 200px;background-color: """+ right_col_colour +""";">{}</td>""".format(bathrooms) + """
        </tr>
        <tr>
        <td style="background-color: """+ left_col_colour +""";"><span style="color: #ffffff;">Home Square Footage</span></td>
        <td style="width: 200px;background-color: """+ right_col_colour +""";">{}</td>""".format(sqft_living) + """
        </tr>
        <tr>
        <td style="background-color: """+ left_col_colour +""";"><span style="color: #ffffff;">Floors</span></td>
        <td style="width: 200px;background-color: """+ right_col_colour +""";">{}</td>""".format(floors) + """
        </tr>
        <tr>
        <td style="background-color: """+ left_col_colour +""";"><span style="color: #ffffff;">Waterfront</span></td>
        <td style="width: 200px;background-color: """+ right_col_colour +""";">{}</td>""".format(waterfront) + """
        </tr>
        <tr>
        <td style="background-color: """+ left_col_colour +""";"><span style="color: #ffffff;">Latitude</span></td>
        <td style="width: 200px;background-color: """+ right_col_colour +""";">{}</td>""".format(latitude) + """
        </tr>
        <tr>
        <td style="background-color: """+ left_col_colour +""";"><span style="color: #ffffff;">Longitude</span></td>
        <td style="width: 200px;background-color: """+ right_col_colour +""";">{}</td>""".format(longitude) + """
        </tr>
        <tr>
        <td style="background-color: """+ left_col_colour +""";"><span style="color: #ffffff;">Derived Price</span></td>
        <td style="width: 200px;background-color: """+ right_col_colour +""";">{}</td>""".format(price) + """
        </tr>
        </tbody>
        </table>
        </html>
        """
    return html

In [4]:
#create lists of input values
zipcodes = [98125, 98136, 98074, 98053, 98003, 98146, 98038, 98115, 98074, 98107, 98126,
              98033, 98070, 98166, 98052, 98178, 98074, 98070, 98040, 98040]
bedrooms = [3, 4, 3, 4, 3, 3, 3, 2, 3, 5, 4, 3, 4, 2, 3, 4, 3, 5, 3, 3]
bathrooms = [2.25, 3.0, 2.0, 4.5, 2.25, 1.0, 2.5, 1.0, 1.75, 2.0, 3.0, 2.0, 1.0,
             1.0, 1.0, 1.75, 2.75, 2.5, 2.25, 2.0]
sqft_living = [2570, 1960, 1680, 5420, 1715, 1780, 1890, 1160, 1370, 1810, 2950,
                 1890, 1600, 1200, 1250, 1620, 3050, 2270, 2450, 1710]
waterfront = [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1]
lat = [47.721, 47.5208, 47.6168, 47.6561, 47.3097, 47.5123, 47.3684, 47.69, 47.6127,
         47.67, 47.5714, 47.4339, 47.4734, 47.6393, 47.5115, 47.645, 47.3853, 47.5779, 47.585, 47.5684]

long = [-122.319, -122.393, -122.045, -122.005, -122.3271, -122.337, -122.031, -122.292, -122.045, -122.394, 
        -122.375, -122.512, -122.365, -122.097, -122.246, -122.084, -122.479, -122.2461, -122.2221, -122.06]
floors =  [1, 2, 1, 1, 1, 2, 2, 1, 1, 1, 2, 2, 2, 3, 1, 2, 3, 1, 2, 1]
year_built = [1951, 1965, 1987, 2001, 1995, 1960, 2003, 1942, 1977, 1989,
              1900, 1943, 1968, 1952, 1999, 1996, 1943, 1928, 1980, 1990]

In [5]:
#convert lists of input values into dicts of single inputs
input_dicts = []
for i in range(20):
    inputs = {
        'Zipcode': zipcodes[i],
        'Bedrooms': bedrooms[i],
        'Bathrooms': bathrooms[i],
        'Home Square Footage': sqft_living[i],
        'Waterfront': waterfront[i],
        'Latitude': lat[i],
        'Longitude': long[i],
        'Floors': floors[i],
        'Year Built': year_built[i]
    }
    input_dicts.append(inputs)

The above two cells create an arbitrary set of inputs to feed into the model.

In [6]:
#run pricing for each input in input_dicts list
output_dicts = []
for inputs in input_dicts:
    
    price = Pricer(model, mapping, bin_config, inputs, binary_vars=['Waterfront'])
    price.run()
    output_dicts.append(price.processed_output)
    print('\n','='*80,'\n')

*WARNING: Input variable 'Latitude' not supported by model!
*WARNING: Input variable 'Longitude' not supported by model!

	== Inputs ==


,Inputs
Category,
Zipcode,98125.000
Bedrooms,3.000
Bathrooms,2.250
Home Square Footage,2570.000
Waterfront,0.000
Latitude,47.721
Longitude,-122.319
Floors,1.000
Year Built,1951.000


Running the model with these inputs, the derived value of the house is: $576886.04


*WARNING: Input variable 'Latitude' not supported by model!
*WARNING: Input variable 'Longitude' not supported by model!

	== Inputs ==


,Inputs
Category,
Zipcode,98136.0000
Bedrooms,4.0000
Bathrooms,3.0000
Home Square Footage,1960.0000
Waterfront,0.0000
Latitude,47.5208
Longitude,-122.3930
Floors,2.0000
Year Built,1965.0000


Running the model with these inputs, the derived value of the house is: $595292.91


*WARNING: Input variable 'Latitude' not supported by model!
*WARNING: Input variable 'Longitude' not supported by model!

	== Inputs ==


...

Running the model with these inputs, the derived value of the house is: $909396.05


*WARNING: Input variable 'Latitude' not supported by model!
*WARNING: Input variable 'Longitude' not supported by model!

	== Inputs ==


,Inputs
Category,
Zipcode,98040.0000
Bedrooms,3.0000
Bathrooms,2.2500
Home Square Footage,2450.0000
Waterfront,1.0000
Latitude,47.5850
Longitude,-122.2221
Floors,2.0000
Year Built,1980.0000


Running the model with these inputs, the derived value of the house is: $1404817.2


*WARNING: Input variable 'Latitude' not supported by model!
*WARNING: Input variable 'Longitude' not supported by model!

	== Inputs ==


,Inputs
Category,
Zipcode,98040.0000
Bedrooms,3.0000
Bathrooms,2.0000
Home Square Footage,1710.0000
Waterfront,1.0000
Latitude,47.5684
Longitude,-122.0600
Floors,1.0000
Year Built,1990.0000


Running the model with these inputs, the derived value of the house is: $1316506.38




The above cell runs each input into the model and displays a dataframe of the inputs as well as a text noting what the model derived prices are. 

In [7]:
#plot each input on a map, enable each marker to have a popup which displays the inputted info
#as well as the model derived price
df = pd.concat(
    [pd.DataFrame(inputs, index=['0']) for inputs in output_dicts], axis=0
).reset_index().drop('index', axis=1)
location = df['Latitude'].mean(), df['Longitude'].mean()
m = folium.Map(location=location,zoom_start=15,min_zoom=5)
for i in range(0, df.shape[0]):
    html = df_to_html(i)
    iframe = branca.element.IFrame(html=html,width=400,height=300)
    popup = folium.Popup(iframe,parse_html=True)
    folium.Marker([df['Latitude'].iloc[i],df['Longitude'].iloc[i]],
                  popup=popup,icon=folium.Icon(icon='info-sign')).add_to(m)

In [8]:
m

The above cells take all the derived model prices and tag them to their respective inputs, and the plots all of these inputs on a Folium map of King County.